In [2]:
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_util

import tensorflow_to_barracuda as tf2bc

In [3]:
action_size = [2]

In [7]:
graph = tf.Graph()
sess = tf.Session(config=generate_session_config(), graph=graph)

with graph.as_default():
    #theses constants are necessary for our model to be interpreted by Barracuda
    tf.Variable(2, name="version_number", trainable=False, dtype=tf.int32)
    tf.Variable(0, name="memory_size", trainable=False, dtype=tf.int32)
    tf.Variable(0, name="is_continuous_control", trainable=False, dtype=tf.int32)
    tf.Variable(sum(action_size), name="action_output_shape", trainable=False, dtype=tf.int32)
    #action_size is an array containing as much elements as there are branches of actions. 
    #Each element describes the number of actions possible for a certain branch.
    
    mask_input = tf.placeholder(shape=[None, 1], dtype=tf.float32, name="action_masks")
    
    action_idx = [0] + list(np.cumsum(action_size))
    branch_masks = [mask_input[:, action_idx[i] : action_idx[i + 1]] for i in range(len(action_size))]
    
    in_ph = tf.placeholder(shape=[None, 1], dtype=tf.float32, name="vector_observation")
    h1_layer = tf.layers.dense(in_ph, 16, None)
    branch1_layer = tf.layers.dense(h1_layer, 2, activation=None) #branch 1 has two actions so we put 2 units at the end of this branch
    #should do a for-loop over action_size, to create X branch layer

    raw_probs = tf.multiply(tf.nn.softmax(branch1_layer), branch_masks)
    normalized_probs = tf.divide(raw_probs, tf.reduce_sum(raw_probs, 1, True))
    normalized_logits = tf.concat(tf.log(normalized_probs), 1)
    normalized_logits = tf.squeeze(normalized_logits, axis=0)
    normalized_logits = tf.identity(normalized_logits, name="action")
    
    init = tf.global_variables_initializer() 

In [8]:
sess.run(init)

In [10]:
settings = SerializationSettings("./models", "Brain name", convert_to_barracuda=True, convert_to_onnx=False)
frozen_graph_def = _make_frozen_graph(settings, graph, sess)

List of nodes to export for brain :Brain name
	version_number
	memory_size
	is_continuous_control
	action_output_shape
	action
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


In [11]:
frozen_graph_def_path = settings.model_path + "/frozen_graph_def.pb"
with gfile.GFile(frozen_graph_def_path, "wb") as f:
    f.write(frozen_graph_def.SerializeToString())

In [12]:
tf2bc.convert(frozen_graph_def_path, settings.model_path + "/model.nn")

Converting ./models/frozen_graph_def.pb to ./models/model.nn
IGNORED: Pack unknown layer
GLOBALS: 'version_number', 'memory_size', 'is_continuous_control', 'action_output_shape'
IN: 'action_masks': [-1, 1, 1, 1] => 'strided_slice'
IN: 'vector_observation': [-1, 1, 1, 1] => 'dense/BiasAdd'
OUT: 'concat/concat', 'action'
DONE: wrote ./models/model.nn file.


In [5]:
from typing import List, NamedTuple

POSSIBLE_OUTPUT_NODES = frozenset(["action", "action_probs", "recurrent_out", "value_estimate"])
MODEL_CONSTANTS = frozenset(["action_output_shape", "is_continuous_control", "memory_size", "version_number"])

class SerializationSettings(NamedTuple):
    model_path: str
    brain_name: str
    convert_to_barracuda: bool = True
    convert_to_onnx: bool = True
    onnx_opset: int = 9

def _make_frozen_graph(settings: SerializationSettings, graph: tf.Graph, sess: tf.Session) -> tf.GraphDef:
    with graph.as_default():
        target_nodes = ",".join(_process_graph(settings, graph))
        graph_def = graph.as_graph_def()
        output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, target_nodes.replace(" ", "").split(","))
    return output_graph_def

def _process_graph(settings: SerializationSettings, graph: tf.Graph) -> List[str]:
    """
    Gets the list of the output nodes present in the graph for inference
    :return: list of node names
    """
    all_nodes = [x.name for x in graph.as_graph_def().node]
    nodes = [x for x in all_nodes if x in POSSIBLE_OUTPUT_NODES | MODEL_CONSTANTS]
    print("List of nodes to export for brain :" + settings.brain_name)
    for n in nodes:
        print("\t" + n)
    return nodes

def generate_session_config() -> tf.ConfigProto:
    """
    Generate a ConfigProto to use for ML-Agents that doesn't consume all of the GPU memory
    and allows for soft placement in the case of multi-GPU.
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    # For multi-GPU training, set allow_soft_placement to True to allow
    # placing the operation into an alternative device automatically
    # to prevent from exceptions if the device doesn't suppport the operation
    # or the device does not exist
    config.allow_soft_placement = True
    return config